<a href="https://colab.research.google.com/github/sanuthit/Risk-Based-Motor-Insurance-Premium-Calculation-System-/blob/risk-model-development/accident_risk_DLmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')
root = "/content/drive/MyDrive"
print("MyDrive exists:", os.path.exists(root))
print("Top folders:", os.listdir(root)[:30])
DATA_DIR = "/content/drive/MyDrive/Data/Datasets"
print("DATA_DIR exists:", os.path.exists(DATA_DIR))
print(os.listdir(DATA_DIR)[:30])

df = pd.read_csv("/content/drive/MyDrive/Data/Datasets/risk_dataset_60000_toyota_suzuki_v2_cleaned.csv", encoding="utf-8")

DATA_PATH = "/content/drive/MyDrive/Data/Datasets/risk_dataset_60000_toyota_suzuki_v2_cleaned.csv"
df = pd.read_csv(DATA_PATH)

print(df.shape)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive exists: True
Top folders: ['Colab Notebooks', 'Data']
DATA_DIR exists: True
['premium_dataset_60000_v3_toyota_suzuki_full.csv', 'risk_dataset_60000.csv', 'risk_dataset_60000_toyota_suzuki_v2_cleaned.csv']
(60000, 49)


,policy_id,customer_id,driver_age,driver_gender,driver_occupation,years_of_driving_experience,member_automobile_assoc_ceylon,has_previous_motor_policy,ncb_percentage,accidents_last_3_years,...,approx_market_value,sum_insured,total_claim_amount_within_1_year,hard_flag_blacklist,driver_age_band,vehicle_age_band,risk_exposure_proxy,doc_missing_score,compliance_risk_score,ncb_validity_flag
0,P000001,C00002,35,M,Accountant,17,1,1,20,1,...,9375583,7691446,0,0,35–44,13+,Low,1,1,0
1,P000002,C00003,40,M,Unemployed,16,0,0,0,0,...,8789777,8210229,0,0,35–44,13+,Low,0,1,0
2,P000003,C00004,33,F,Businessman,8,0,1,10,4,...,5143262,4628639,680769,0,25–34,13+,Low,1,0,0
3,P000004,C00005,45,F,Farmer,27,0,1,35,0,...,7518522,7142596,0,0,45–59,13+,High,0,0,0
4,P000005,C00006,51,F,Businessman,18,0,1,10,1,...,6677872,6343978,0,0,45–59,4–7,High,1,1,0


In [5]:

RISK_FEATURES = [
    # Driver risk
    "driver_age",
    "driver_age_band",
    "driver_gender",
    "driver_occupation",
    "years_of_driving_experience",
    "member_automobile_assoc_ceylon",

    # Driving & claim history (inputs only)
    "has_previous_motor_policy",
    "accidents_last_3_years",
    "ncb_percentage",

    # Vehicle risk
    "vehicle_type",
    "vehicle_segment",
    "engine_capacity_cc",
    "fuel_type",
    "vehicle_age_years",
    "vehicle_age_band",
    "has_lpg_conversion",

    # Usage & exposure
    "vehicle_usage_type",
    "risk_exposure_proxy",
    "registration_district",
    "parking_type",

    # Behavioural / compliance proxy (optional but allowed)
    "doc_missing_score",
    "compliance_risk_score"
]


In [6]:
TARGET = "had_claim_within_1_year"

In [7]:
df_risk = df[RISK_FEATURES + [TARGET]].copy()

print(df_risk.shape)
df_risk.head()


(60000, 23)


,driver_age,driver_age_band,driver_gender,driver_occupation,years_of_driving_experience,member_automobile_assoc_ceylon,has_previous_motor_policy,accidents_last_3_years,ncb_percentage,vehicle_type,...,vehicle_age_years,vehicle_age_band,has_lpg_conversion,vehicle_usage_type,risk_exposure_proxy,registration_district,parking_type,doc_missing_score,compliance_risk_score,had_claim_within_1_year
0,35,35–44,M,Accountant,17,1,1,1,20,Car,...,13,13+,0,Private,Low,Jaffna,Street,1,1,0
1,40,35–44,M,Unemployed,16,0,0,0,0,Car,...,22,13+,0,Private,Low,Kandy,Garage,0,1,0
2,33,25–34,F,Businessman,8,0,1,4,10,Car,...,21,13+,0,Private,Low,Colombo,Street,1,0,1
3,45,45–59,F,Farmer,27,0,1,0,35,SUV,...,14,13+,0,Hire,High,Kandy,Garage,0,0,0
4,51,45–59,F,Businessman,18,0,1,1,10,Car,...,7,4–7,0,Hire,High,Galle,Garage,1,1,0


In [8]:
X = df_risk[RISK_FEATURES]
y = df_risk[TARGET]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (60000, 22)
y shape: (60000,)


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Use the already-created df_risk (raw features + target)
train_df, temp_df = train_test_split(
    df_risk,
    test_size=0.30,
    stratify=df_risk[TARGET],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.50,
    stratify=temp_df[TARGET],
    random_state=42
)

print(train_df.shape, val_df.shape, test_df.shape)
print(train_df[TARGET].value_counts(normalize=True))


(42000, 23) (9000, 23) (9000, 23)
had_claim_within_1_year
0    0.859833
1    0.140167
Name: proportion, dtype: float64


In [10]:
categorical_cols = [
    "driver_age_band",
    "driver_gender",
    "driver_occupation",
    "vehicle_type",
    "vehicle_segment",
    "fuel_type",
    "vehicle_age_band",
    "vehicle_usage_type",
    "risk_exposure_proxy",
    "registration_district",
    "parking_type",
]

continuous_cols = [
    "driver_age",
    "years_of_driving_experience",
    "member_automobile_assoc_ceylon",
    "has_previous_motor_policy",
    "accidents_last_3_years",
    "ncb_percentage",
    "engine_capacity_cc",
    "vehicle_age_years",
    "has_lpg_conversion",
    "doc_missing_score",
    "compliance_risk_score",
]


In [11]:
train_df[continuous_cols] = train_df[continuous_cols].astype("float32")
val_df[continuous_cols]   = val_df[continuous_cols].astype("float32")
test_df[continuous_cols]  = test_df[continuous_cols].astype("float32")

train_df[categorical_cols] = train_df[categorical_cols].astype("category")
val_df[categorical_cols]   = val_df[categorical_cols].astype("category")
test_df[categorical_cols]  = test_df[categorical_cols].astype("category")

# 01. FT-Transformer

In [11]:
!pip install pytorch-tabular torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successful

In [12]:
import torch
from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.ft_transformer.config import FTTransformerConfig

In [13]:
import torch
from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.ft_transformer.config import FTTransformerConfig

data_config = DataConfig(
    target=[TARGET],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

trainer_config = TrainerConfig(
    max_epochs=50,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    batch_size=1024,
    early_stopping="valid_loss",
    early_stopping_patience=10,
    load_best=False
)

optimizer_config = OptimizerConfig()

model_config = FTTransformerConfig(
    task="classification",
    learning_rate=1e-3,
    num_heads=8,
    num_attn_blocks=4,
    attn_dropout=0.2,
    ff_dropout=0.2,
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    trainer_config=trainer_config,
    optimizer_config=optimizer_config,
)

tabular_model.fit(train=train_df, validation=val_df)


Epoch 11/49 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42/42 0:01:36 • 0:00:00 0.46it/s v_num: 0.000 train_loss: 0.425     
                                                                                valid_loss: 0.395 valid_accuracy:  
                                                                                0.860 train_accuracy: 0.860        

INFO:pytorch_tabular.tabular_model:Training the model completed


In [15]:
import numpy as np
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, precision_recall_curve

# ✅ Define test inputs/labels from your split dataframes
X_test = test_df[RISK_FEATURES].copy()
y_test = test_df[TARGET].astype(int).values

# ✅ Predict on test set
pred_df = tabular_model.predict(X_test)

print("Prediction columns:", list(pred_df.columns))

# ✅ Find the probability column safely (different versions name it differently)
prob_candidates = [c for c in pred_df.columns if "probability" in c.lower()]

if len(prob_candidates) == 0:
    raise ValueError(f"No probability column found. Columns are: {pred_df.columns}")

# Prefer positive-class probability if available
preferred = [
    f"{TARGET}_1_probability",              # common in some classification outputs
    f"{TARGET}_probability",                # common in binary classification outputs
]

prob_col = next((c for c in preferred if c in pred_df.columns), prob_candidates[0])
print("Using probability column:", prob_col)

proba = pred_df[prob_col].values

# ✅ ROC-AUC
auc = roc_auc_score(y_test, proba)
print("TEST ROC-AUC:", auc)

# ✅ Best threshold by F1 (from Precision-Recall curve)
precision, recall, thresholds = precision_recall_curve(y_test, proba)
f1 = (2 * precision * recall) / (precision + recall + 1e-9)

best_idx = np.argmax(f1[:-1])   # last point has no threshold
best_thr = thresholds[best_idx]

print("Best threshold:", best_thr)
print("Best F1:", f1[best_idx])
print("Precision:", precision[best_idx], "Recall:", recall[best_idx])

# ✅ Predictions with tuned threshold
y_pred = (proba >= best_thr).astype(int)

print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification report:")
print(classification_report(y_test, y_pred, digits=4))

/usr/local/lib/python3.12/dist-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.12/dist-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

Prediction columns: ['had_claim_within_1_year_0_probability', 'had_claim_within_1_year_1_probability', 'had_claim_within_1_year_prediction']
Using probability column: had_claim_within_1_year_1_probability
TEST ROC-AUC: 0.6335087257460615
Best threshold: 0.1712371
Best F1: 0.29255505033795737
Precision: 0.18762609280430398 Recall: 0.6637589214908802

Confusion matrix:
[[4115 3624]
 [ 424  837]]

Classification report:
              precision    recall  f1-score   support

           0     0.9066    0.5317    0.6703      7739
           1     0.1876    0.6638    0.2926      1261

    accuracy                         0.5502      9000
   macro avg     0.5471    0.5977    0.4814      9000
weighted avg     0.8059    0.5502    0.6174      9000



# 02. NODE

In [16]:
import pytorch_tabular.models.node.config as node_cfg

# show anything that looks like a config class
[name for name in dir(node_cfg) if "Config" in name or "config" in name]


['ModelConfig', 'NodeConfig']

In [17]:
import inspect
print(inspect.getsource(node_cfg)[:1500])

import warnings
from dataclasses import dataclass, field
from typing import Optional

from pytorch_tabular.config import ModelConfig


@dataclass
class NodeConfig(ModelConfig):
    """Neural Oblivious Decision Ensembles for Deep Learning on Tabular Data configuration.

    Args:
        num_layers (int): Number of Oblivious Decision Tree Layers in the Dense Architecture

        num_trees (int): Number of Oblivious Decision Trees in each layer

        additional_tree_output_dim (int): The additional output dimensions which is only used to pass
                through different layers of the architectures. Only the first output_dim outputs will be used for
                prediction

        depth (int): The depth of the individual Oblivious Decision Trees

        choice_function (str): Generates a sparse probability distribution to be used as feature
                weights(aka, soft feature selection). Choices are: [`entmax15`,`sparsemax`].

        bin_function (str): Generates a s

In [18]:
from pytorch_tabular.models.node.config import NodeConfig

In [19]:
from sklearn.model_selection import train_test_split

X = df_risk[RISK_FEATURES]
y = df_risk[TARGET]

# 70% Train, 30% Temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

# Split temp into 15% Validation, 15% Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)


Train: (42000, 22) (42000,)
Val:   (9000, 22) (9000,)
Test:  (9000, 22) (9000,)


In [20]:
from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.node.config import NodeConfig
import torch

data_config = DataConfig(
    target=[TARGET],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

trainer_config = TrainerConfig(
    max_epochs=50,
    accelerator="cpu",          # GPU not available
    devices=1,
    early_stopping="valid_loss",
    early_stopping_patience=10,
    batch_size=1024,
    load_best=False,
)

optimizer_config = OptimizerConfig()

model_config = NodeConfig(
    task="classification",
    learning_rate=1e-3,
    num_layers=2,               # keep small on CPU
    num_trees=256,              # reduce from default
    depth=4,
    input_dropout=0.1,
    choice_function="entmax15",
    bin_function="entmoid15",
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    trainer_config=trainer_config,
    optimizer_config=optimizer_config,
)

tabular_model.fit(train=train_df, validation=val_df)


INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
/usr/local/lib/python3.12/dist-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.12/dist-packages/pytorch_tabular/categorical_encoders.py

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ NODEBackbone     │  1.4 M │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │    279 │ train │
│ 2 │ _head            │ Lambda           │      0 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 1.4 M                                                                                            
Non-trainable params: 258                                                                                          
Total params: 1.4 M                                                                                                
Total estimated model params size (MB): 5                                                                          
Modules in train mode: 21                                                                                          
Modules in eval mode: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches 
(42) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if
you want to see logs for the training epoch.

INFO:pytorch_tabular.tabular_model:Training the model completed


In [21]:
import torch
print(torch.cuda.is_available(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

False CPU


In [23]:
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix
import numpy as np

# --- Validation probabilities ---
val_pred_df = tabular_model.predict(val_df)
val_prob_col = [c for c in val_pred_df.columns if "probability" in c.lower()][0]
val_proba_ft = val_pred_df[val_prob_col].values

print("Validation ROC-AUC:", roc_auc_score(y_val, val_proba_ft))

# --- Find best threshold on validation set ---
thresholds = np.arange(0.05, 0.95, 0.01)
f1s = [f1_score(y_val, (val_proba_ft >= t).astype(int)) for t in thresholds]
best_t = thresholds[np.argmax(f1s)]

print("Best threshold:", best_t)

# --- Test probabilities ---
test_pred_df = tabular_model.predict(test_df)
test_prob_col = [c for c in test_pred_df.columns if "probability" in c.lower()][0]
test_proba_ft = test_pred_df[test_prob_col].values

# --- Apply best threshold on test ---
test_pred = (test_proba_ft >= best_t).astype(int)

print("TEST ROC-AUC:", roc_auc_score(y_test, test_proba_ft))
print(confusion_matrix(y_test, test_pred))
print(classification_report(y_test, test_pred, digits=4))


/usr/local/lib/python3.12/dist-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.12/dist-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

Validation ROC-AUC: 0.3778562194762792
Best threshold: 0.6600000000000001


/usr/local/lib/python3.12/dist-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
/usr/local/lib/python3.12/dist-packages/pytorch_tabular/categorical_encoders.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

TEST ROC-AUC: 0.3714947177846964
[[   1 7738]
 [   1 1260]]
              precision    recall  f1-score   support

           0     0.5000    0.0001    0.0003      7739
           1     0.1400    0.9992    0.2456      1261

    accuracy                         0.1401      9000
   macro avg     0.3200    0.4997    0.1229      9000
weighted avg     0.4496    0.1401    0.0346      9000



# 03. TabNet